
# <center>NumPy案例</center>

### 实践题目
#### 泰坦尼克乘客数据分析生还率

* 著名的数据分析竞赛网站Kaggle上，举行了很多数据分析比赛，其中比较著名的就有 泰坦尼克号乘客生还预测 。
* Kaggle提供的数据集中，共有1309名乘客数据，其中891是已知存活情况，剩下418则是需要进行分析预测的。

### 数据文件
* titanic-data.csv

### 本节任务
* 分析探索原始数据，查看数据空值等情况
* 构建特征工程，统计特征分布与生还关系

数据主要字段说明如下：
* PassengerId: 乘客编号
* Survived: Survived (1) or died (0)是否存活
* Pclass: 船舱
* Name: 姓名
* Sex: 性别
* Age: 年龄
* SibSp: 兄弟/姐妹/配偶的数量
* Parch: 父母/子女的数量
* Ticket: 票号
* Fare: 票价
* Cabin: 座号
* Embarked: 登船港口

引入文件操作所需的包


In [1]:
import os
import numpy as np

In [2]:
#在这里直接读取首行的时候会出现换行符 \n 所以利用切边给它过滤掉，通过字符串分割转换成列表
filename="titanic-data.csv"  #文件在代码的当前目录，如果不在需要给出全路径
with open(filename,encoding='utf-8') as f:
    head_index=np.array(f.readline()[:-1].split(','))  #第一行是列名,多个函数通过.连起来，顺序执行
head_index

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='<U11')

In [3]:
data = np.loadtxt('titanic-data.csv', delimiter=',',dtype=np.str, skiprows=1) 
#从文件中加载数据直接生成array
data

array([['1', '0', '3', ..., '7.25', '', 'S'],
       ['2', '1', '1', ..., '71.2833', 'C85', 'C'],
       ['3', '1', '3', ..., '7.925', '', 'S'],
       ...,
       ['889', '0', '3', ..., '23.45', '', 'S'],
       ['890', '1', '1', ..., '30', 'C148', 'C'],
       ['891', '0', '3', ..., '7.75', '', 'Q']], dtype='<U83')

In [4]:
# 查看首行数据
first_line = data[0]
first_line

array(['1', '0', '3', '"Braund Mr. Owen Harris"', 'male', '22', '1', '0',
       'A/5 21171', '7.25', '', 'S'], dtype='<U83')

In [6]:
# 有效字段索引
a = np.arange(12)
data_index = np.delete(a, 3)  # name字段的值对生还率预测没有意义，可以删除
data_index

array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11])

In [7]:
# 去除name索引数据，通过整数数组来取ndarray的数据
head_index = head_index[data_index]
head_index

array(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='<U11')

In [8]:
# 去除name那一列数据
corpus_data = data[:,data_index]  #数组切片，冒号表示所有行都取，data_index是取的列的编号列表
corpus_data

array([['1', '0', '3', ..., '7.25', '', 'S'],
       ['2', '1', '1', ..., '71.2833', 'C85', 'C'],
       ['3', '1', '3', ..., '7.925', '', 'S'],
       ...,
       ['889', '0', '3', ..., '23.45', '', 'S'],
       ['890', '1', '1', ..., '30', 'C148', 'C'],
       ['891', '0', '3', ..., '7.75', '', 'Q']], dtype='<U83')

### 缺失值分析

Age和Cabin字段有较多缺失值，Embanked字段有2个缺失值

In [9]:
items_data = corpus_data.T  #矩阵转置， 旋转90度，原来每一列 成为 新矩阵的每一行
items_data.shape

(11, 891)

In [10]:
null_data = []
for col in items_data:    #对于 items_data每一行 复制给变量col
    null_data.append(sum(col==''))   #计算这一行空值个数
null_data

for i in range(0,head_index.size):
    print('%s 空值个数： %s' % (head_index[i], null_data[i]))

PassengerId 空值个数： 0
Survived 空值个数： 0
Pclass 空值个数： 0
Sex 空值个数： 0
Age 空值个数： 177
SibSp 空值个数： 0
Parch 空值个数： 0
Ticket 空值个数： 0
Fare 空值个数： 0
Cabin 空值个数： 687
Embarked 空值个数： 2


处理缺失值

需要进行缺失值处理的有：Age、Cabin、Embarked



In [11]:
items_data.shape

(11, 891)

In [12]:
# age属性中有缺失, 通过计算该属性的均值将缺失处填补,使得数据的数量一致
age_index = 4
print(head_index.shape)
print(age_index)

print("age_index:%d"%age_index) 
age_data = items_data[age_index]
print(age_data)
age_data_null = sum(age_data=='')
handle_data = np.where(age_data == '', 0, age_data) #dkdkfdkkaa
age_mean = sum(handle_data.astype(np.float))/(len(handle_data)-age_data_null) #非空的年龄的平均值
print(age_mean)

(11,)
4
age_index:4
['22' '38' '26' '35' '35' '' '54' '2' '27' '14' '4' '58' '20' '39' '14'
 '55' '2' '' '31' '' '35' '34' '15' '28' '8' '38' '' '19' '' '' '40' '' ''
 '66' '28' '42' '' '21' '18' '14' '40' '27' '' '3' '19' '' '' '' '' '18'
 '7' '21' '49' '29' '65' '' '21' '28.5' '5' '11' '22' '38' '45' '4' '' ''
 '29' '19' '17' '26' '32' '16' '21' '26' '32' '25' '' '' '0.83' '30' '22'
 '29' '' '28' '17' '33' '16' '' '23' '24' '29' '20' '46' '26' '59' '' '71'
 '23' '34' '34' '28' '' '21' '33' '37' '28' '21' '' '38' '' '47' '14.5'
 '22' '20' '17' '21' '70.5' '29' '24' '2' '21' '' '32.5' '32.5' '54' '12'
 '' '24' '' '45' '33' '20' '47' '29' '25' '23' '19' '37' '16' '24' '' '22'
 '24' '19' '18' '19' '27' '9' '36.5' '42' '51' '22' '55.5' '40.5' '' '51'
 '16' '30' '' '' '44' '40' '26' '17' '1' '9' '' '45' '' '28' '61' '4' '1'
 '21' '56' '18' '' '50' '30' '36' '' '' '9' '1' '4' '' '' '45' '40' '36'
 '32' '19' '19' '3' '44' '58' '' '42' '' '24' '28' '' '34' '45.5' '18' '2'
 '32' '26' '16' '40'

In [13]:
items_data[4] = np.where(items_data[4] == '', age_mean, items_data[4])  #数组赋值为数组

In [14]:
#embarked填充众数
embarked_index = 10
embarked_data = items_data[embarked_index ]
embarked_data

array(['S', 'C', 'S', 'S', 'S', 'Q', 'S', 'S', 'S', 'C', 'S', 'S', 'S',
       'S', 'S', 'S', 'Q', 'S', 'S', 'C', 'S', 'S', 'Q', 'S', 'S', 'S',
       'C', 'S', 'Q', 'S', 'C', 'C', 'Q', 'S', 'C', 'S', 'C', 'S', 'S',
       'C', 'S', 'S', 'C', 'C', 'Q', 'S', 'Q', 'Q', 'C', 'S', 'S', 'S',
       'C', 'S', 'C', 'S', 'S', 'C', 'S', 'S', 'C', '', 'S', 'S', 'C',
       'C', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'C', 'S', 'S', 'S', 'S',
       'S', 'S', 'S', 'S', 'Q', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S',
       'S', 'S', 'S', 'S', 'S', 'C', 'C', 'S', 'S', 'S', 'S', 'S', 'S',
       'S', 'S', 'S', 'S', 'S', 'Q', 'S', 'C', 'S', 'S', 'C', 'S', 'Q',
       'S', 'C', 'S', 'S', 'S', 'C', 'S', 'S', 'C', 'Q', 'S', 'C', 'S',
       'C', 'S', 'S', 'S', 'S', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'S',
       'Q', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'C',
       'Q', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S',
       'S', 'S', 'Q', 'S', 'S', 'C', 'S', 'S', 'C', 'S', 'S', 'S'

In [15]:
np.unique(embarked_data)

array(['', 'C', 'Q', 'S'], dtype='<U83')

In [16]:
from collections import  Counter
Counter(embarked_data)  #统计不同舱位数量

Counter({'S': 644, 'C': 168, 'Q': 77, '': 2})

解释：np.where(condition, x, y)
函数参数：满足条件(condition)，输出x，不满足输出y， 对array每个数据迭代处理，最后输出还是一个array

In [17]:
items_data[10] = np.where(items_data[10] == '', 'S', items_data[10])  #填充缺失值为 众数

In [18]:
head_index = np.delete(head_index,9)   #座号 Cablin 可以删掉这一行
items_data = np.delete(items_data,9, axis = 0)

### 数据离散化
```
# 获取sex的值, 并用0和1代表男性和女性
np.unique(items_data[3])
items_data[3] = np.where(items_data[3] == 'male', 0, items_data[3])
items_data[3] = np.where(items_data[3] == 'female', 1, items_data[3])

# 获取embarked的值, 用0,1,2分别表示S,C,Q
np.unique(items_data[9])
items_data[9] = np.where(items_data[9] == 'S', 0, items_data[9])
items_data[9] = np.where(items_data[9] == 'C', 1, items_data[9])
items_data[9] = np.where(items_data[9] == 'Q', 2, items_data[9])
```

In [19]:
np.unique(items_data[3])

array(['female', 'male'], dtype='<U83')

In [20]:
items_data[3] = np.where(items_data[3] == 'male', 0, items_data[3])
items_data[3] = np.where(items_data[3] == 'female', 1, items_data[3])

In [21]:
np.unique(items_data[9])
items_data[9] = np.where(items_data[9] == 'S', 0, items_data[9])
items_data[9] = np.where(items_data[9] == 'C', 1, items_data[9])
items_data[9] = np.where(items_data[9] == 'Q', 2, items_data[9])

### 数据分析1
```
# 生还者总数
items_data[1].astype(np.int32).sum(axis=0)
# 平均生还率
items_data[1].astype(np.int32).mean(axis=0)
```
结论1：生还者总数为342人，平均生还率约为38%

In [22]:
head_index

array(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked'], dtype='<U11')

In [23]:
items_data[1].astype(np.int32).sum()    #一维数组 axis只有0，可以不写

342

In [24]:
items_data[1].astype(np.int32).mean()  # 342/总人数

0.3838383838383838

### 数据分析2
```
# 各等级船舱Pclass乘客总数量统计：
from collections import  Counter
data = np.array(items_data[2])
Counter(data)

# 各等级船舱生还数量统计：'Pclass', 'Survived'
pclass_survived = np.vstack((items_data[2],items_data[1]))
unique, counts = np.unique(pclass_survived, return_counts=True,axis=1)

# 各船舱等级对应的平均生还率
```

结论2：

头等舱总人数为216人，二等舱为184人，三等舱为491人。

头等舱生还人数为136人，二等舱为87人，三等舱为119人。

头等舱生还率为62.96%，二等舱为47.28%，三等舱为24.24%。

In [25]:
from collections import  Counter
data = np.array(items_data[2])
Counter(data)

Counter({'3': 491, '1': 216, '2': 184})

In [26]:
pclass_survived = np.vstack((items_data[2],items_data[1]))  #两个数组 垂直方向连接， 本案例就是两行数据 并起来

In [27]:
pclass_survived

array([['3', '1', '3', ..., '3', '1', '3'],
       ['0', '1', '1', ..., '0', '1', '0']], dtype='<U83')

In [28]:
unique, counts = np.unique(pclass_survived, return_counts=True,axis=1) # 垂直方向的组合 统计不同组合个数

In [29]:
unique

array([['1', '1', '2', '2', '3', '3'],
       ['0', '1', '0', '1', '0', '1']], dtype='<U83')

In [30]:
counts

array([ 80, 136,  97,  87, 372, 119])

In [31]:
a = [216, 184, 491]
b = [136, 87, 119]
np.array(b)/np.array(a)  #不同舱位生还率

array([0.62962963, 0.47282609, 0.24236253])

### 编程练习

模仿数据分析2， 计算不同性别平均生还率 ['Sex','Survived']



In [32]:
from collections import  Counter
data = np.array(items_data[2])
Counter(data)



Counter({'3': 491, '1': 216, '2': 184})

In [60]:
male = []
female = []
for i in range(len(items_data[3])):
    if items_data[3][i] == '0':
        male.append(int(items_data[1][i]))
    else:
        female.append(int(items_data[1][i]))

In [61]:
print(sum(male)/len(male), sum(female)/len(female))

0.18890814558058924 0.7420382165605095
